# Reactions
This notebook contains all the steps followed in order to reconcile the existing metabolic reconstruction in CHO cells. It si divided into two parts: **1. Network Reconstruction** and **2. Identification of duplicated reactions**

[1. Network Reconstruction](#reconstruction) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.1 Datasets generation and merge**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.2 Normalization of the data** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.3 Group all the data into a unified dataset** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction** <br>
&nbsp;&nbsp;&nbsp;&nbsp;**1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets** <br>

[2. Identification of Duplicated Reactions](#duplicated) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Generate the model and identify duplicated reactions**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above** <br>

[3. Add BiGG database links to every reaction](#bigg) <br>

[4. Divide into compartments](#compartments) <br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.1 Generate the model and identify duplicated reactions**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above** <br>

## 1. Network Reconstruction <a id='reconstruction'></a>
These include the generation of datasets (1), normalization of the data (2), merging all four reconstructions into a unified dataset (3), compiling the generated reconstruction into a cobra model in order to identify duplicated reactions (4), fixing duplicated reactions using BiGG ID annotation (5), adding Recon3D GPR information (6), and finally dividing the dataset into two different datasets that will be further curated in Google Sheets (7).

In [ ]:
# Libraries import
import pandas as pd
import numpy as np

### 1.1 Datasets generation and merge
Dataset generation from previous reconstructions (CHO_DG44, iCHO1766, iCHO2101, iCHO2291)

In [ ]:
#Read excel files and create the dfs

#Camels CHO_DG44 metabolic reconstruction
camel_df = pd.read_excel('../Data/Reconciliation/datasets/CHO_DG44.xlsx', header = 1)

#Hefzi's iCHO1766 metabolic reconstruction
hefzi_df = pd.read_excel('../Data/Reconciliation/datasets/hefzi_final.xlsx')

#Foudaliha's iCHO2101 metabolic reconstruction
fouladiha_df = pd.read_excel('../Data/Reconciliation/datasets/iCHO2101.xlsx', 'Supplementary Table 10', header = 1)

#Yeo's iCHO2291 metabolic reconstruction
iCHO2291 = pd.read_excel('../Data/Reconciliation/datasets/iCHO2291_final.xlsx')

### 1.2 Normalization of the data
All dataset are normalized into the same shape and format and then combined into one big dataset

In [ ]:
#Standarization of the columns names
camel_df.rename(columns = {'Reaction ID':'Reaction', 'Initial reaction in model':'Reaction Formula', 'Reaction name':'Reaction Name', 'Justification':'Curation Notes'}, inplace = True)
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula', 'GPR':'GPR_fou'}, inplace = True)

# Addition of tag columns for organizational purposes
camel_df.insert(loc=0, column='cam', value='X')
camel_df.insert(loc=1, column='hef', value=np.nan)
camel_df.insert(loc=2, column='fou', value=np.nan)
camel_df.insert(loc=3, column='yeo', value=np.nan)

hefzi_df.insert(loc=0, column='cam', value=np.nan)
hefzi_df.insert(loc=1, column='hef', value='X')
hefzi_df.insert(loc=2, column='fou', value=np.nan)
hefzi_df.insert(loc=3, column='yeo', value=np.nan)

fouladiha_df.insert(loc=0, column='cam', value=np.nan)
fouladiha_df.insert(loc=1, column='hef', value=np.nan)
fouladiha_df.insert(loc=2, column='fou', value='X')
fouladiha_df.insert(loc=3, column='yeo', value=np.nan)

iCHO2291.insert(loc=0, column='cam', value=np.nan)
iCHO2291.insert(loc=1, column='hef', value=np.nan)
iCHO2291.insert(loc=2, column='fou', value=np.nan)
iCHO2291.insert(loc=3, column='yeo', value='X')

In [ ]:
# Generate cols list with the column names from all datasets
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+iCHO2291.columns.to_list()+camel_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

In [ ]:
def add_col(df):
    '''
    This function adds the columns from the cols list 
    that are not present in the df
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [ ]:
# Unify columns for all datasets
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
iCHO2291 = add_col(iCHO2291)
camel_df = add_col(camel_df)

In [ ]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
iCHO2291 = iCHO2291[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
hefzi_df = hefzi_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
camel_df = camel_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR_hef','GPR_fou','GPR_yeo', 'Subsystem', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]

camel_df['Reaction'] = camel_df['Reaction'].str.strip()

In [ ]:
# Merge all the dfs into a unified df
all_dfs = pd.concat([camel_df, hefzi_df, fouladiha_df, iCHO2291])
all_dfs = all_dfs.reset_index(drop = True)

#Unify reaction names
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace('_cho', '')
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace(r"(e)", "_e_", regex = False)
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("[", "_")
all_dfs['Reaction'] = all_dfs['Reaction'].str.replace("]", "_")

all_dfs #20940 rows/reactions (many of them repeated)

In [ ]:
# Remove the underscore at the end of some reactions

rxns = []
idx = []

for i,row in all_dfs.iterrows():
    if str(row['Reaction']).endswith('_'):
        s = re.sub('_$', '', row['Reaction'])
        rxns.append(s)
        idx.append(i)
        
all_dfs['Reaction'].update(pd.Series(rxns,index=idx))

### 1.3 Group all the data into a unified dataset
The combined dataset generated above "all_dfs" is grouped by the Reaction BiGG ID to obtain a dataset with unique reaction identifiers

In [ ]:
# Group the data into a unified dataset

all_dfs2 = all_dfs.groupby('Reaction').first()
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace('[','_')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(']','')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' => | =>',' --> ')
all_dfs2['Reaction Formula'] = all_dfs2['Reaction Formula'].str.replace(' <-- | <--',' <=> ')
all_dfs2

In [ ]:
# Separate Demand Reaccion into a different df and remove from reconstruction. 
# Keep demand reactions from extracellular space

all_dfs2 = all_dfs2.reset_index()
demand_reactions = pd.DataFrame(columns = all_dfs2.columns)

for index, rxn in all_dfs2.iterrows():
    rxn = rxn['Reaction Formula']
    a,b = re.split('<=>|-->',rxn)
    if (b == '' or b == ' ') and not a.endswith('_e '):
        temp_df = all_dfs2[all_dfs2['Reaction Formula'] == rxn]
        demand_reactions = pd.concat([demand_reactions,temp_df])
        all_dfs2.drop(index, inplace=True)

demand_reactions = demand_reactions.reset_index(drop=True)
demand_reactions.to_excel('../Data/Reconciliation/datasets/demand_reactions.xlsx')
demand_reactions

In [ ]:
all_dfs2 = all_dfs2.reset_index(drop=True)
all_dfs2

### 1.4 Addition of the Recon3D CHO ortholog GPRs into the reconstruction
The information from the dataset generated in the notebook "GPR Annotation" containing all the information from Recon3D GPRS in human and its corresponding CHO orthologs is mapped into our reconstruction

In [ ]:
# Generate a dict with recon3d reactions as keys and the CHO GPRs as values.
recon3d = pd.read_excel('../Data/GPR_Curation/recon3D_chogprs.xlsx')
recon3d_dict = recon3d.set_index('m_reaction')['CHO GPR'].to_dict()
recon3d_dict

In [ ]:
# Map 'recon3d_dict' into the all_dfs3 dataframe
# the reaction IDs should be the same as those in our reconstruction

all_dfs3 = all_dfs2.reset_index()
all_dfs3['GPR_Recon3D'] = all_dfs3['Reaction'].map(recon3d_dict)

### 1.5 Divide the dataset in two to facilitate reading and curation in Google Sheets
The dataset all_dfs3 is divided into two different datasets: all_dfs4 contains mainly all the information regarding GPRs assigned from previous reconstructions. all_dfs5 contains the rest of the attributes in the reconstruction such as EC number, bounds, etc.

In [ ]:
# all_dfs4 contains mainly information of the reactions GPRs
all_dfs4 = all_dfs3[['Reaction', 'Reaction Name', 'Reaction Formula', 'Subsystem', 'GPR_hef', 'GPR_fou', 'GPR_yeo', 'GPR_Recon3D', 'Curation Notes', 'References']]
all_dfs4.insert(8,'GPR_final', '')
all_dfs4.to_excel('../Data/Reconciliation/datasets/all_dfs4.xlsx')
all_dfs4

In [ ]:
# all_dfs5 contains information of the rest of the attributes in our reconstruction
all_dfs5 = all_dfs3[['Reaction', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible', 'Lower bound', 'Upper bound', 'Objective']]
all_dfs5.to_excel('../Data/Reconciliation/datasets/all_dfs5.xlsx')
all_dfs5

## 2. Identifications of Duplicated Reactions <a id='duplicated'></a>
In this part of the notebook we use the **duplicated_reactions** fucntion from the **utils** module to spot duplicated reactions in our dataset. First we generate a model, using cobprapy, with our dataset, then we apply the **duplicated_reactions** function to the model, and finally we standarize the name of the duplicated reactions according to the nomenclature used in BiGG.

In [ ]:
# Libraries import
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from tqdm.notebook import tqdm
from cobra import Model, Reaction, Metabolite

from utils import duplicated_reactions
from google_sheet import GoogleSheet

### 2.1 Generate the model and identify duplicated reactions
Generation of a cobra model from the **Google Sheet dataset**. This model will be used to identify duplicated reactions from the stoichiometric matrix of the model.

In [ ]:
# Generate the necessary datasets for the identification of the duplicated reactions
KEY_FILE_PATH = 'credentials.json'
SPREADSHEET_ID_v3 = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID_v3, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_rxns = 'Rxns'
sheet_attributes = 'Attributes'

reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

In [ ]:
##### ----- Create a model from the reactions sheet ----- #####
model = Model("iCHO")
lr = []
for _, row in reactions.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)    
model.add_reactions(lr)
model

In [ ]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(tqdm(model.reactions)):
    print(r.id)
    r.build_reaction_from_string(reactions['Reaction Formula'][i])
    r.name = reactions['Reaction Name'][i]
    r.subsystem = reactions['Subsystem'][i]
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])

In [ ]:
duplicated_reactions = duplicated_reactions(model)
duplicated_reactions

### 2.2 Fix duplicated reactions in the dataset from the list duplicated_reactions obtained above
Duplicated reactions are itereated in a for loop and mapped in the original dataset. A request is made in the 
BiGG database http://bigg.ucsd.edu with each of the duplicated reactions. If any of the duplicated reactions is in BiGG, the other reaction automatically changes its name to the one located in Bigg. This way we unifiy the names of our reactions to those in BiGG.

In [ ]:
i = 0
for rxn in tqdm(duplicated_reactions):
    
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[0]])
    if response.status_code == 200:
        if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
            print(f'1 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} present in CHOv1 model')
            reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
            rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
            i += 1
    
    elif response.status_code != 200:
        response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+reactions['Reaction'][rxn[1]])
        if response.status_code == 200:
            if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                print(f'2 Reaction {reactions.iloc[rxn[0],2]} changed for {reactions.iloc[rxn[1],2]} present in CHOv1 model')
                reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                i += 1
            
        elif response.status_code != 200:
            response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[0]])
            if response.status_code == 200:
                if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                    print(f'3 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} present BiGG database')
                    reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                    rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                    i += 1
                
            elif response.status_code != 200:
                response=session.get('http://bigg.ucsd.edu/universal/reactions/'+reactions['Reaction'][rxn[1]])
                if response.status_code == 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'4 Reaction {reactions.iloc[rxn[0],2]} changed for {reactions.iloc[rxn[1],2]} present BiGG database')
                        reactions.loc[rxn[0], 'Reaction'] = reactions['Reaction'][rxn[1]]
                        rxns_attributes.loc[rxn[0], 'Reaction'] = rxns_attributes['Reaction'][rxn[1]]
                        i += 1
                elif response.status_code != 200:
                    if reactions['Reaction'][rxn[0]] == rxns_attributes['Reaction'][rxn[0]] and reactions.loc[rxn[1], 'Reaction'] == rxns_attributes.loc[rxn[1], 'Reaction']:
                        print(f'5 Reaction {reactions.iloc[rxn[1],2]} changed for {reactions.iloc[rxn[0],2]} not present in Bigg DB')
                        reactions.loc[rxn[1], 'Reaction'] = reactions['Reaction'][rxn[0]]
                        rxns_attributes.loc[rxn[1], 'Reaction'] = rxns_attributes['Reaction'][rxn[0]]
                        i += 1

print(len(duplicated_reactions))
print(i)

In [ ]:
# Store the original column order
column_order_rxns = reactions.columns.tolist()
column_order_att = rxns_attributes.columns.tolist()

# Group by 'BiGG ID' and keep the first non-null value in each group, then reset the index
reactions = reactions.groupby('Reaction').first().reset_index()
rxns_attributes = rxns_attributes.groupby('Reaction').first().reset_index()

# Rearrange the columns to the original order
reactions = reactions[column_order_rxns]
rxns_attributes = rxns_attributes[column_order_att]

In [ ]:
##############################################################
#### ---------------------------------------------------- ####
#### ---- Update Rxns and  Attributes Google Sheets ----- ####
#### ---------------------------------------------------- ####
##############################################################
sheet.update_google_sheet(sheet_rxns, reactions)
sheet.update_google_sheet(sheet_attributes, rxns_attributes)
print("Google Sheet updated.")

In [ ]:
# Check that Reactions IDs and formulas are equal in "Rxns" and "Attributes" sheets
reactions = sheet.read_google_sheet(sheet_rxns)
rxns_attributes = sheet.read_google_sheet(sheet_attributes)

rxnsIDseq = list(reactions['Reaction']) == list(rxns_attributes['Reaction'])
if rxnsIDseq:
    print('Reaction IDs in the Rxns and Attributes Sheets are equal\n')
else:
    rxns_sheet_ids = set(list(reactions['Reaction']))
    attr_sheet_ids = set(list(rxns_attributes['Reaction']))
    print(f'Reaction IDs that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction IDs that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

rxnsforseq = list(reactions['Reaction Formula']) == list(rxns_attributes['Reaction Formula'])
if rxnsforseq:
    print('Reaction Formulas in the Rxns and Attributes Sheets are equal')
else:
    rxns_sheet_forms = set(list(reactions['Reaction Formula']))
    attr_sheet_forms = set(list(rxns_attributes['Reaction Formula']))
    print(f'Reaction formulas that are in Rxns Sheet and not in Attributes Sheet {rxns_sheet_forms - attr_sheet_forms}\n')
    print(f'Reaction formulas that are in Attributes Sheet and not in Rxns Sheet {attr_sheet_forms - rxns_sheet_forms}\n')

## 3. Add BiGG database links to every reaction <a id='bigg'></a>
Here we identify which reactions from our dataset are located in BiGG and we add a direct link to that reaction in our dataset

In [1]:
# Libraries import
import time
import requests
from requests_html import HTMLSession

from utils import duplicated_reactions
from google_sheet import GoogleSheet

In [2]:
# Generate the necessary datasets
KEY_FILE_PATH = 'credentials.json'
SPREADSHEET_ID_v3 = '1MlBXeHIKw8k8fZyXm-sN__AHTRSunJxar_-bqvukZws'

# Initialize the GoogleSheet object
sheet = GoogleSheet(SPREADSHEET_ID_v3, KEY_FILE_PATH)

# Read data from the Google Sheet
sheet_rxns = 'Rxns'
reactions = sheet.read_google_sheet(sheet_rxns)

In [3]:
in_chov1=0
in_bigg=0
not_bigg=0

session = HTMLSession()

# A function to handle requests and retries
def fetch_url(url, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            response = session.get(url)
            if response.status_code == 200:
                return response
            else:
                retries += 1
        except requests.exceptions.RequestException as e:
            print(f"Error occurred: {e}, retrying...")
            retries += 1
            time.sleep(2)  # wait for 2 seconds before next retry
    return None

for i,row in reactions.iterrows():
    rxn=row['Reaction']
    
    response_esp = fetch_url('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn)
    
    if response_esp is not None:
        print(rxn,'in CHOv1')
        reactions.loc[i, 'BiGG database'] = 'http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn
        in_chov1+=1
    else:
        response_gen = fetch_url('http://bigg.ucsd.edu/universal/reactions/'+rxn)
        if response_gen is not None:
            print(rxn,'in Bigg')
            reactions.loc[i, 'BiGG database'] = 'http://bigg.ucsd.edu/universal/reactions/'+rxn
            in_bigg+=1
        else:
            print(rxn,'not in Bigg')
            not_bigg+=1

10FTHF5GLUtl in CHOv1
10FTHF5GLUtm in CHOv1
10FTHF6GLUtl in CHOv1
10FTHF6GLUtm in CHOv1
10FTHF7GLUtl in CHOv1
10FTHF7GLUtm in CHOv1
10FTHFtl in CHOv1
10FTHFtm in CHOv1
11DOCRTSLte in Bigg
11DOCRTSLtm in CHOv1
11DOCRTSLtr in CHOv1
11DOCRTSTRNte in Bigg
11DOCRTSTRNtm in CHOv1
11DOCRTSTRNtr in CHOv1
12DHCHOLabc in Bigg
12DHCHOLt in Bigg
12DHCHOLt2 in Bigg
12HPETATP in Bigg
12HPETUPKt in Bigg
13DAMPPOX in CHOv1
13DAMPPte in Bigg
15HPETATP in Bigg
15HPETUPKt in Bigg
15KPROSTGF2c in Bigg
15KPROSTGF2t in Bigg
17AHPRGNLONEte in Bigg
17AHPRGSTRNte in Bigg
1A2425THVITD2t not in Bigg
1A2425THVITD2tm not in Bigg
1A25DHVITD3te in Bigg
1MNCAMti in CHOv1
1PPDCRp in CHOv1
1a25DHVITD3TRn in CHOv1
1a_24_25VITD2Hm in CHOv1
1a_24_25VITD3Hm in CHOv1
1a_25VITD2Hm in CHOv1
1a_25VITD3Hm in CHOv1
21HPRGNLONE in Bigg
23PDE7pl not in Bigg
24NPHte in CHOv1
24_25DHVITD2t in CHOv1
24_25DHVITD2tm in CHOv1
24_25DHVITD3t in CHOv1
24_25DHVITD3tm in CHOv1
24_25VITD2Hm in CHOv1
24_25VITD3Hm in CHOv1
25HVITD2t in CHOv1
25

ACACT143n3p not in Bigg
ACACT150m not in Bigg
ACACT161n9m not in Bigg
ACACT162n6em not in Bigg
ACACT163n3m not in Bigg
ACACT163n3p not in Bigg
ACACT170m not in Bigg
ACACT182n6Em not in Bigg
ACACT183n3m not in Bigg
ACACT183n3p not in Bigg
ACACT183n6m not in Bigg
ACACT184n3m not in Bigg
ACACT184n3p not in Bigg
ACACT1r in CHOv1
ACACT1rm in Bigg
ACACT1x in CHOv1
ACACT201n9p not in Bigg
ACACT204n3m not in Bigg
ACACT204n3p not in Bigg
ACACT204n6m not in Bigg
ACACT204n6p not in Bigg
ACACT205n3m not in Bigg
ACACT205n3p not in Bigg
ACACT221n9p not in Bigg
ACACT224n6m not in Bigg
ACACT224n6p not in Bigg
ACACT2251n3m not in Bigg
ACACT2251n3p not in Bigg
ACACT226n3m not in Bigg
ACACT226n3p not in Bigg
ACACT241n9p not in Bigg
ACACT244n6m not in Bigg
ACACT244n6p not in Bigg
ACACT245n6m not in Bigg
ACACT245n6p not in Bigg
ACACT246n3p not in Bigg
ACACT260p not in Bigg
ACACT4p in CHOv1
ACACT50m not in Bigg
ACACT5p in CHOv1
ACACT6p in CHOv1
ACACT70m not in Bigg
ACACT7p in CHOv1
ACACT81n3m not in Bigg
AC

ALKP in CHOv1
ALLTTtm in Bigg
ALOX12 in CHOv1
ALOX12R in CHOv1
ALOX15 in CHOv1
ALOX5 in CHOv1
ALOX52 in CHOv1
ALPAP not in Bigg
ALR in CHOv1
ALR2 in CHOv1
ALR3 in CHOv1
AM6SAD in CHOv1
AMACR2p in CHOv1
AMACR2r in Bigg
AMACRDMHPm not in Bigg
AMACRDMNm not in Bigg
AMACRPRISTp not in Bigg
AMACRTMHDp not in Bigg
AMACRTMTDp not in Bigg
AMACRp in CHOv1
AMACRr in Bigg
AMANK in CHOv1
AMCOXO in CHOv1
AMETr in CHOv1
AMETt2m in CHOv1
AMETtd in Bigg
AMETtn in CHOv1
AMPDA in CHOv1
AMPTASECG in CHOv1
AMPTASECGe in CHOv1
AMPt4 not in Bigg
AMPtp in CHOv1
AMPtr in CHOv1
AMY1e in CHOv1
AMY2e in CHOv1
AND19ONEc in Bigg
ANDRSTANDRte in Bigg
ANDRSTNDNte in Bigg
ANDRSTRNGLCte in CHOv1
ANDRSTRNGLCtr in CHOv1
ANDRSTRNte in CHOv1
ANDRSTRNtr in CHOv1
ANTHte in CHOv1
ANTIPYRENEte in CHOv1
AOBUTDsm in CHOv1
AP4AH1 in CHOv1
APAT2rm in CHOv1
APNNOXte in CHOv1
APOCF in CHOv1
APOCFm in CHOv1
APOC_LYS_BTNP in CHOv1
APOC_LYS_BTNPm in CHOv1
APPNNte in CHOv1
APRGSTRNt1e in Bigg
APRGSTRNte in CHOv1
APRTO2 in CHOv1
AQCOBAL

CA24GSr in Bigg
CA24Gte in Bigg
CA2te not in Bigg
CA2tm not in Bigg
CA3Sabc in Bigg
CA3St in Bigg
CAATPS in CHOv1
CAATPaser in CHOv1
CAATPasg not in Bigg
CACNAO in CHOv1
CACNAO2 in CHOv1
CAMPt in CHOv1
CARNMT not in Bigg
CARNS1 in CHOv1
CAROtr in CHOv1
CARPEPT1tc in CHOv1
CARVEOLte in CHOv1
CARhPTtc in CHOv1
CASULT in Bigg
CAT in CHOv1
CAT2p in CHOv1
CATm in CHOv1
CATp in CHOv1
CAt7r in CHOv1
CAtm7 not in Bigg
CBASPte in CHOv1
CBL1tm not in Bigg
CBL2OR in CHOv1
CBL2tm in CHOv1
CBLATm in CHOv1
CBLTDe in CHOv1
CBLtle not in Bigg
CBPPer in CHOv1
CBPS in CHOv1
CBPSam in CHOv1
CBPter in CHOv1
CBR1 in CHOv1
CBR2 in CHOv1
CCA_D3t in CHOv1
CCA_D3tm in CHOv1
CDCA24GSc in Bigg
CDCA24GSr in Bigg
CDCA24Gte in Bigg
CDCA3GSc in Bigg
CDCA3GSr in Bigg
CDCA3Gte in Bigg
CDCA6AH in Bigg
CDIPTr in Bigg
CDPDAGtm in Bigg
CDPRBTLtg not in Bigg
CDPtg not in Bigg
CDS in Bigg
CDSm in Bigg
CE1273t1 in Bigg
CE1352te in Bigg
CE1447te in Bigg
CE1918te in Bigg
CE2006te in Bigg
CE2026te in Bigg
CE2209te in Bigg
CE221

DESAT20_5 not in Bigg
DESAT22_1p in CHOv1
DESAT22_2p in CHOv1
DESAT24_1 in CHOv1
DESAT24_51 not in Bigg
DESAT24_6 not in Bigg
DGAT in Bigg
DGCHOLt2e not in Bigg
DGCHOLte in CHOv1
DGCHOLtx in CHOv1
DGK1 in CHOv1
DGK2m in CHOv1
DGNSKm in CHOv1
DGSNt in CHOv1
DGSNtm in CHOv1
DGTPtm in Bigg
DGTPtn in CHOv1
DGULND in CHOv1
DHAAt1r in CHOv1
DHAK in CHOv1
DHAPA in Bigg
DHAPAx in Bigg
DHAPtc in Bigg
DHCHOLESTANATEATP in Bigg
DHCHOLESTANATEt in Bigg
DHCHOLESTANATEtm in CHOv1
DHCR241r in CHOv1
DHCR242r in CHOv1
DHCR243r in CHOv1
DHCR71r in CHOv1
DHCR72r in CHOv1
DHCRD1 in Bigg
DHCRD2 in Bigg
DHDASCBtm in CHOv1
DHDDH in CHOv1
DHDDS in CHOv1
DHDPBMTm in CHOv1
DHEASABCCte in Bigg
DHEASULT in CHOv1
DHEASt in CHOv1
DHEAStr in CHOv1
DHEAtr in CHOv1
DHFR in CHOv1
DHFRx not in Bigg
DHFtl in CHOv1
DHFtm in CHOv1
DHORD9 in CHOv1
DHORTS in CHOv1
DHPM1 in CHOv1
DHPM2 in CHOv1
DHPR in CHOv1
DHPR2 in CHOv1
DHPS in Bigg
DHRT_ibcoa in CHOv1
DIDECAETH in Bigg
DIDP in CHOv1
DIDPtn in CHOv1
DIGALSGALSIDEATPte in B

...

S6T18g in CHOv1
S6T19g in CHOv1
S6T1g in CHOv1
S6T20g in CHOv1
S6T21g in CHOv1
S6T22g in CHOv1
S6T23g in CHOv1
S6T24g in CHOv1
S6T25g in CHOv1
S6T2g in CHOv1
S6T3g in CHOv1
S6T4g in CHOv1
S6T5g in CHOv1
S6T6g in CHOv1
S6T7g in CHOv1
S6T8g in CHOv1
S6T9g in CHOv1
S6TASE10ly in CHOv1
S6TASE11ly in CHOv1
S6TASE12ly in CHOv1
S6TASE13ly in CHOv1
S6TASE14ly in CHOv1
S6TASE15ly in CHOv1
S6TASE16ly in CHOv1
S6TASE17ly in CHOv1
S6TASE18ly in CHOv1
S6TASE19ly in CHOv1
S6TASE1ly in CHOv1
S6TASE20ly in CHOv1
S6TASE21ly in CHOv1
S6TASE22ly in CHOv1
S6TASE23ly in CHOv1
S6TASE24ly in CHOv1
S6TASE25ly in CHOv1
S6TASE26ly in CHOv1
S6TASE2ly in CHOv1
S6TASE3ly in CHOv1
S6TASE4ly in CHOv1
S6TASE5ly in CHOv1
S6TASE6ly in CHOv1
S6TASE7ly in CHOv1
S6TASE8ly in CHOv1
S6TASE9ly in CHOv1
SACCD3m in CHOv1
SACCD4m in CHOv1
SADT in CHOv1
SADTn in CHOv1
SALMCOM in CHOv1
SALMCOM2 in CHOv1
SAMHISTA in CHOv1
SAMHISTAe in Bigg
SARCOXp in CHOv1
SARCStex in CHOv1
SARCStm in CHOv1
SARCStp in CHOv1
SARDHm in CHOv1
SAT1 in

SameRxns128 not in Bigg
SameRxns129 not in Bigg
SameRxns130 not in Bigg
SameRxns131 not in Bigg
SameRxns132 not in Bigg
SameRxns133 not in Bigg
SameRxns135 not in Bigg
SameRxns136 not in Bigg
SameRxns137 not in Bigg
SameRxns138 not in Bigg
SameRxns139 not in Bigg
SameRxns140 not in Bigg
SameRxns141 not in Bigg
SameRxns142 not in Bigg
SameRxns143 not in Bigg
SameRxns145 not in Bigg
SameRxns146 not in Bigg
SameRxns147 not in Bigg
SameRxns148 not in Bigg
SameRxns149 not in Bigg
SameRxns150 not in Bigg
SameRxns151 not in Bigg
SameRxns152 not in Bigg
SameRxns153 not in Bigg
SameRxns154 not in Bigg
SameRxns155 not in Bigg
SameRxns156 not in Bigg
SameRxns157 not in Bigg
SameRxns158 not in Bigg
SameRxns159 not in Bigg
SameRxns160 not in Bigg
SameRxns161 not in Bigg
SameRxns162 not in Bigg
SameRxns163 not in Bigg
SameRxns164 not in Bigg
SameRxns165 not in Bigg
SameRxns166 not in Bigg
SameRxns167 not in Bigg
Ser_Thrtg in CHOv1
T2M26DCOAHLm in CHOv1
T2M26DCOAHLx in CHOv1
T4HCINNMFM in CHOv1
T4HCI

VALTRAHm not in Bigg
VALTRPPHEr in Bigg
VALTRPPHEt in Bigg
VALTRPVALr in Bigg
VALTRPVALt in Bigg
VALTRS in Bigg
VALTRSm in Bigg
VALVALr in Bigg
VALVALt in Bigg
VALt4 in CHOv1
VALt5m in CHOv1
VALt7l not in Bigg
VALtec in CHOv1
VANILPYRc in Bigg
VD3 in CHOv1
VITD2Hm in CHOv1
VITD2t in CHOv1
VITD2tm in CHOv1
VITD3Hm in CHOv1
VITD3t in CHOv1
VITD3t2 in Bigg
VITD3tm in CHOv1
VITEtl in CHOv1
VITKtl in CHOv1
VLCS2p in CHOv1
VLCS2r in CHOv1
VLCSp in CHOv1
VLCSr in CHOv1
WHARACHDtr in Bigg
WHDDCAte in CHOv1
WHHDCAte in CHOv1
WHTSTSTERONEte in CHOv1
WHTSTSTERONEtr in Bigg
WHTTDCAte in CHOv1
XAND in CHOv1
XANDp in CHOv1
XANtx in CHOv1
XAO in CHOv1
XAO2 in CHOv1
XAO2x in CHOv1
XAOx in CHOv1
XOL27OHtm in CHOv1
XOL7AH2tm in CHOv1
XOL7AH2tr in CHOv1
XOL7AH3ATP in Bigg
XOL7AH3t in Bigg
XOL7AONEATP in Bigg
XOL7AONEt in Bigg
XOL7AONEtr in CHOv1
XOLDIOLONEATP in Bigg
XOLDIOLONEt in CHOv1
XOLDIOLONEtm in CHOv1
XOLEST181CEH not in Bigg
XOLEST181te not in Bigg
XOLEST181tl not in Bigg
XOLEST182_HSte in Bigg


r1095 not in Bigg
r1096 not in Bigg
r1097 not in Bigg
r1098 not in Bigg
r1099 not in Bigg
r1100 not in Bigg
r1101 not in Bigg
r1103 not in Bigg
r1104 not in Bigg
r1105 not in Bigg
r1106 in CHOv1
r1109 in CHOv1
r1112 not in Bigg
r1113 not in Bigg
r1116 in CHOv1
r1117 in CHOv1
r1129 in CHOv1
r1135 in Bigg
r1143 in CHOv1
r1144 in Bigg
r1147 in Bigg
r1148 in CHOv1
r1150 in CHOv1
r1154 in Bigg
r1155 in CHOv1
r1156 in Bigg
r1159 in CHOv1
r1162 in CHOv1
r1163 in Bigg
r1164 in CHOv1
r1165 in CHOv1
r1166 in Bigg
r1167 in CHOv1
r1168 in CHOv1
r1169 in CHOv1
r1170 in CHOv1
r1171 in CHOv1
r1172 in CHOv1
r1173 in CHOv1
r1174 in CHOv1
r1175 in Bigg
r1176 not in Bigg
r1177 in CHOv1
r1178 in CHOv1
r1179 in CHOv1
r1180 in CHOv1
r1181 in CHOv1
r1182 in CHOv1
r1183 in CHOv1
r1184 in CHOv1
r1185 not in Bigg
r1186 not in Bigg
r1187 not in Bigg
r1188 not in Bigg
r1189 not in Bigg
r1190 not in Bigg
r1251 in CHOv1
r1252 in CHOv1
r1253 in Bigg
r1254 in Bigg
r1255 in CHOv1
r1257 in CHOv1
r1259 in CHOv1
r1260 in

r1942 in CHOv1
r1943 in CHOv1
r1944 in CHOv1
r1945 in CHOv1
r1946 in CHOv1
r1947 in CHOv1
r1948 in CHOv1
r1949 in CHOv1
r1950 in CHOv1
r1951 in CHOv1
r1952 in CHOv1
r1953 in CHOv1
r1954 in CHOv1
r1955 in CHOv1
r1956 in CHOv1
r1957 in CHOv1
r1958 in CHOv1
r1959 in CHOv1
r1960 in CHOv1
r1961 in CHOv1
r1962 in CHOv1
r1963 in CHOv1
r1964 in CHOv1
r1965 in CHOv1
r1966 in CHOv1
r1967 in CHOv1
r1968 in CHOv1
r1969 in CHOv1
r1970 in CHOv1
r1971 in CHOv1
r1972 in CHOv1
r1973 in CHOv1
r1974 in CHOv1
r1975 in CHOv1
r1976 in CHOv1
r1977 in CHOv1
r1978 in CHOv1
r1979 in CHOv1
r1980 in CHOv1
r1981 in CHOv1
r1982 in CHOv1
r1983 in CHOv1
r1984 in CHOv1
r1985 in CHOv1
r1986 in CHOv1
r1987 in CHOv1
r1988 in CHOv1
r1989 in CHOv1
r1990 in CHOv1
r1991 in CHOv1
r1992 in CHOv1
r1993 in CHOv1
r1994 in CHOv1
r1995 in CHOv1
r1996 in CHOv1
r1997 in CHOv1
r1998 in CHOv1
r1999 in CHOv1
r2000 in CHOv1
r2001 in CHOv1
r2002 in CHOv1
r2003 in CHOv1
r2004 in CHOv1
r2005 in CHOv1
r2006 in CHOv1
r2007 in CHOv1
r2008 in C

In [4]:
print(f'Reactions from CHOv1 reconstruction {in_chov1}')
print(f'Reactions in BiGG database {in_bigg} [not including those in CHOv1)')
print(f'Reactions NOT in BiGG {not_bigg}')

Reactions from CHOv1 reconstruction 5330
Reactions in BiGG database 3187 [not including those in CHOv1)
Reactions NOT in BiGG 1989


In [5]:
reactions

,Package,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,Conf. Score,Curation Notes,References,BiGG database
0,7,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,,http://bigg.ucsd.edu/models/iCHOv1/reactions/1...
1,7,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,,http://bigg.ucsd.edu/models/iCHOv1/reactions/1...
2,7,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,,http://bigg.ucsd.edu/models/iCHOv1/reactions/1...
3,7,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,1,No information available in the literature abo...,,http://bigg.ucsd.edu/models/iCHOv1/reactions/1...
4,7,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,1,No information available in the literature abo...,,http://bigg.ucsd.edu/models/iCHOv1/reactions/1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,22,,r2534,Major Facilitator(MFS) TCDB:2.A.1.44.1,thr_L_e <=> thr_L_c,"TRANSPORT, EXTRACELLULAR",,,100757617,100757617,100757617,,,,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...
10502,22,,r2535,Major Facilitator(MFS) TCDB:2.A.1.44.1,hom_L_e <=> hom_L_c,TRANSPORT,,,100757617,100757617,100757617,,,,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...
10503,22,,r2537,Utilized transport,lnlncgcoa_c <=> lnlncgcoa_r,TRANSPORT,,,,,,,,,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...
10504,22,,r2538,Utilized transport,dlnlcgcoa_c <=> dlnlcgcoa_r,TRANSPORT,,,,,,,,,http://bigg.ucsd.edu/models/iCHOv1/reactions/r...


In [6]:
#############################################
#### ----------------------------------- ####
#### ---- Update Rxns Google Sheet ----- ####
#### ----------------------------------- ####
#############################################
sheet.update_google_sheet(sheet_rxns, reactions)
print("Google Sheet updated.")

Google Sheet updated.


## 4. Divide into compartments <a id='compartments'></a>

In [ ]:
from cobra.io import load_json_model
import pandas as pd

# model = read_sbml_model('iCHOv3_CHO_05072023.xml')
model = load_json_model('iCHOv3_CHO_05072023.json')

In [ ]:
subsystem_list = []
for r in model.reactions:
    subsystem_list.append(r.subsystem)
print("Number of unique subsystems:", len(set(subsystem_list)))

In [ ]:
subsystem_counts = pd.Series(subsystem_list).value_counts().reset_index()
subsystem_counts.columns = ['Subsystem', 'RXN Counts']
subsystem_counts[subsystem_counts['RXN Counts'] == 1]


In [ ]:
subsystem_counts

In [ ]:
# TODO -Remove blocked reactions from packages
#      -Define the high-importance reactions for biomass production
#      -Rename subsystems with very few reactions